# Advent of Code 2022

I liked [Peter Norvig's approach](https://github.com/norvig/pytudes/blob/main/ipynb/Advent-2020.ipynb) [last year](https://github.com/codemonkeyjim/adventofcode-2021/blob/main/aoc-2021.ipynb), so I'm going to use it again this year.

## Day 0: Imports and Utility Functions
Preparations prior to Day 1:

- Some imports.
- A way to read the day's data file and to print/check the output.
- Some utilities that are likely to be useful.


In [ ]:
from __future__ import annotations
from collections import Counter, defaultdict, namedtuple
from dataclasses import dataclass
from functools import reduce
from itertools import accumulate, chain, islice, permutations, zip_longest
from math import prod
import operator
from queue import PriorityQueue
from statistics import mean, median
from typing import Callable


In [ ]:
def data(day: int, parser=str, sep='\n', filetype="input") -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    sections = open(f'data/advent2022/{filetype}{day}.txt').read().rstrip().split(sep)
    return [parser(section) for section in sections]
     
def do(day, *answers) -> dict[int, int]:
    "E.g., do(3) returns {1: day3_1(in3), 2: day3_2(in3)}. Verifies `answers` if given."
    g = globals()
    got = []
    for part in (1, 2):
        fname = f'day{day}_{part}'
        if fname in g: 
            got.append(g[fname](g[f'in{day}']))
            if len(answers) >= part: 
                assert got[-1] == answers[part - 1], (
                    f'{fname}(in{day}) got {got[-1]}; expected {answers[part - 1]}')
    return got

def by_line(text: str) -> list[str]:
    "Split the text into a list of lines."
    return text.strip().splitlines()

def first(iterable, default=None) -> object:
    "Return first item in iterable, or default."
    return next(iter(iterable), default)

def rest(sequence) -> object: return sequence[1:]

def take_n(iterable, n=1, fillvalue = None):
  slices = (islice(iterable, i, None, n) for i in range(n))
  return zip_longest(*slices, fillvalue = fillvalue)

## Day 1: Calorie Counting

1. Find the Elf carrying the most Calories. How many total Calories is that Elf carrying?
2. Find the top three Elves carrying the most Calories. How many Calories are those Elves carrying in total?

In [ ]:
in1 = data(1, parser=lambda lines: [int(line) for line in lines.split("\n")], sep="\n\n")

In [ ]:
def day1_1(packs: list(int)) -> int:
    return max([sum(pack) for pack in packs])

In [ ]:
def day1_2(packs: list(int)) -> int:
    return sum(sorted([sum(pack) for pack in packs])[-3:])

In [ ]:
do(1, 69626)

## Day 2: Rock Paper Scissors

1. What would your total score be if everything goes exactly according to your strategy guide?

In [ ]:
in2 = data(2, parser=lambda line: line.split(" "))

In [ ]:
OPPONENT_MAP = {
    'A': 0,
    'B': 1,
    'C': 2,
}

MY_MAP = {
    'X': 0,
    'Y': 1,
    'Z': 2,
}

OUTCOME_SCORE = [
    3, # Tie
    6, # Win
    0, # Loss
]

Hand = list[str]

In [ ]:
def hand_score(hand: Hand, opponent_map:dict[str, int]=OPPONENT_MAP, my_map: dict[str, int]=MY_MAP) -> int:
    opponent_play = opponent_map[hand[0]]
    my_play = my_map[hand[1]]

    round_score = OUTCOME_SCORE[(my_play - opponent_play) % 3]
    return round_score + my_play + 1

assert hand_score(['A', 'Y']) == 8
assert hand_score(['B', 'X']) == 1
assert hand_score(['C', 'Z']) == 6


In [ ]:
def day2_1(hands: list[Hand]) -> int:
    return sum([hand_score(hand) for hand in hands])

Don't get fancy. The number of options is small enough to hardcode a table of hands: what to play to get the specified outcome.

In [ ]:
FIX_HAND = {
    'A': {'X': 'Z', 'Y': 'X', 'Z': 'Y'},
    'B': {'X': 'X', 'Y': 'Y', 'Z': 'Z'},
    'C': {'X': 'Y', 'Y': 'Z', 'Z': 'X'},
}
def fix_hand(hand: Hand) -> Hand:
    return [hand[0], FIX_HAND[hand[0]][hand[1]]]

In [ ]:
def day2_2(hands: list[Hand]) -> int:
    return day2_1([fix_hand(hand) for hand in hands])

In [ ]:
do(2, 11603)

## Day 3: Rucksack Reorganization

1. Find the item type that appears in both compartments of each rucksack. What is the sum of the priorities of those item types?

In [ ]:
Sack = tuple[set[str], set[str]]

def compartmentalize(sack: str) -> Sack:
    mid = len(sack) // 2
    return (set(sack[:mid]), set(sack[mid:]))

assert compartmentalize("abcdefgh") == ({'a', 'b', 'c', 'd'}, {'e', 'f', 'g', 'h'})

def both_sides(sack: Sack) -> str:
    items = list(sack[0].intersection(sack[1]))
    assert len(items) == 1
    return items[0]

def whole_sack(sack: Sack) -> set[str]:
    return sack[0].union(sack[1])

def common_item(sacks: list[Sack]) -> str:
    items = list(reduce(lambda intersection, sack: intersection.intersection(sack), sacks))
    assert len(items) == 1
    return items[0]

item_priorities = {**{chr(val): val - ord('a') + 1 for val in range(ord('a'), ord('z')+1)}, **{chr(val): val - ord('A') + 27 for val in range(ord('A'), ord('Z')+1)}}

In [ ]:
in3 = data(3, parser=compartmentalize)

In [ ]:
def day3_1(sacks: list[Sack]) -> int:
    return sum([item_priorities[both_sides(sack)] for sack in sacks])

In [ ]:
def day3_2(sacks: list[Sack]) -> int:
    return sum([item_priorities[common_item(list(map(whole_sack, elf_group)))] for elf_group in take_n(sacks, 3)])


In [ ]:
do(3, 7848, 2616)